## 概要
  * 作成日 : 2020/3/10（火）
  * 作成者 : 藤根
  
## アプローチ内容
  * チームに関する以下データを整形し、学習に利用する。
    1. 所属カンファレンス名
    1. コーチ名
    1. NCAA出場年数
    1. シード
    1. 各シーズンにおける累計得点
  * 以下の手法で学習・予測する。
    1. LogisticRegression、RandomForest、SVCなどの複数アルゴリズムを適用したモデルを作る
    1. グリッドサーチで、各モデルの最適なパラメータを探索する
    1. 各モデルをアンサンブルし、複数の予測結果から最終的な予測結果（もしくは予測確率）を取得する
    1. データを訓練／テスト用に分けず、全データで訓練し、全データで予測する（当然、学習済みのデータに対して予測するため、スコアは高くなるはず）
  
## 結果
  * スコアは0.445 -> 0.30018(predict_proba-RandomForestClassifier.csv)
  * randomforestが今回のデータに適しているというよりも、過学習によってスコアが伸びたようにしか見えない。
  * 一方で疑問もある。このコード上はrandomforestのaccuracyが1.0なのに、提出時のスコアが0にならないのはなぜだろうか？
    * 予測すべきチームの組合せが漏れている？
  
## 次のアプローチ
  * ここから更にスコアを上げることは、過学習をさらに深めることと同義である。
  * よって、Stage-1のアプローチはこの辺で止めた方が良いかもしれない。
  * やるとしたら、ErrorやWarningを取り除くくらいか。

In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
FILEDIR = Path('/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament')

In [ ]:
# 提出用ファイルを取得
sub = pd.read_csv(FILEDIR / 'MSampleSubmissionStage1_2020.csv', usecols=['ID'])
id_splited = sub['ID'].str.split('_', expand=True).astype(int).rename(columns={0: 'Season', 1: 'Team1', 2: 'Team2'})
sub = pd.concat([sub, id_splited], axis=1).set_index(['Season', 'Team1', 'Team2']).sort_index()

In [ ]:
# シーズン毎の出場チームを抽出
tourney_teams = {}
tourney_teams_all = set()
for season in sub.index.get_level_values('Season').drop_duplicates():
    tourney_teams[season] = set()
    tourney_teams[season].update(sub.loc[season].index.get_level_values('Team1'))
    tourney_teams[season].update(sub.loc[season].index.get_level_values('Team2'))
    tourney_teams_all.update(tourney_teams[season])
{k: len(v) for k, v in tourney_teams.items()}

In [ ]:
# 所属カンファレンス情報を取得
conferences = pd.read_csv(FILEDIR / 'MDataFiles_Stage1/MTeamConferences.csv')
conferences = pd.concat(
    [conferences.query('Season == @season and TeamID in @teams') for season, teams in tourney_teams.items()])
conferences = conferences.set_index(['Season', 'TeamID']).sort_index()

In [ ]:
# コーチ名を取得
coaches = pd.read_csv(FILEDIR / 'MDataFiles_Stage1/MTeamCoaches.csv')
coaches = pd.concat(
    [coaches.query('Season == @season and TeamID in @team') for season, team in tourney_teams.items()])
coaches = coaches[coaches['LastDayNum'] == 154].set_index(['Season', 'TeamID']).sort_index()[['CoachName']]

In [ ]:
# NCAAの初回出場年を取得し、初回出場年から現在までの年数を計算
teams = pd.read_csv(FILEDIR / 'MDataFiles_Stage1/MTeams.csv', usecols=['TeamID', 'FirstD1Season'])
teams['FirstD1Season'] = 2020 - teams['FirstD1Season']
teams = pd.concat(
    [teams.query('TeamID in @team').assign(Season=season) for season, team in tourney_teams.items()])
teams = teams.set_index(['Season', 'TeamID']).sort_index()

In [ ]:
# 各シーズンのシードを取得
seeds = pd.read_csv(FILEDIR / 'MDataFiles_Stage1/MNCAATourneySeeds.csv')
seeds = pd.concat(
    [seeds.query('Season == @season and TeamID in @teams') for season, teams in tourney_teams.items()])
seeds = seeds.set_index(['Season', 'TeamID']).sort_index()
seeds['Region'] = seeds['Seed'].str[0]
seeds['Number'] = seeds['Seed'].str[1:3].astype(int)
del seeds['Seed']

In [ ]:
# レギュラーシーズンの累計得点と累計失点を取得
regular = pd.read_csv(FILEDIR / 'MDataFiles_Stage1/MRegularSeasonDetailedResults.csv')
regular = regular.drop(columns=['DayNum', 'LTeamID'])
regular = pd.concat(
    [regular.query('Season == @season and WTeamID in @teams') for season, teams in tourney_teams.items()])
regular = regular.groupby(['Season', 'WTeamID']).sum()
regular = regular.rename_axis(index=['Season', 'TeamID'])

In [ ]:
# 上記取得データをindexで結合
ctcsr = pd.concat([coaches, teams, conferences, seeds, regular], axis=1)

In [ ]:
# NCAAMトーナメントの勝敗結果を取得
result = pd.read_csv(FILEDIR / 'MDataFiles_Stage1/MNCAATourneyCompactResults.csv')
result = result[result['Season'] >= 2015].set_index(['Season', 'WTeamID', 'LTeamID'])

In [ ]:
# 各種データと勝敗結果を結合
merged_teams = pd.concat(
    [ctcsr.loc[[(season, wteam), (season, lteam)], :] for season, wteam, lteam, in result.index])

team1 = merged_teams.iloc[::2, :].reset_index('TeamID') # teams winned
team2 = merged_teams.iloc[1::2, :].reset_index('TeamID') # teams losed

merged_teams = pd.concat([
    pd.concat([team1.add_suffix('1'), team2.add_suffix('2')], axis=1).assign(Res=1),
    pd.concat([team2.add_suffix('1'), team1.add_suffix('2')], axis=1).assign(Res=0),
]).reset_index().set_index(['Season', 'TeamID1', 'TeamID2']).sort_index()

In [ ]:
# 結合データから、説明変数Xを抽出
# Xを数値化、正規化
x_columns = merged_teams.columns[merged_teams.columns != 'Res']
X = merged_teams[x_columns]

for column in X.select_dtypes(include='number'):
    X[column] = MinMaxScaler().fit_transform(X[column].to_numpy().reshape(-1,1))  # FIXME: SettingWithCopyWarning occurred
X = pd.get_dummies(X, columns=x_columns[X.dtypes == 'object'])

In [ ]:
# 目的変数yを設定
y = merged_teams['Res']

In [ ]:
# 学習
clfs = {}

# SVC
clfs['SVC'] = {
    'instance': SVC(probability=True),
    'params': [
        {'kernel': ['linear'], 'C': [0.01, 0.05, 0.1, 0.5, 1]},
        {'kernel': ['rbf'], 'C': [1, 10, 50, 100, 250], 'gamma': [0.1, 0.2, 0.3]}
    ]    
}

# RandomForest
clfs['RandomForestClassifier'] = {
    'instance': RandomForestClassifier(n_jobs=-1),
    'params': {        
        'n_estimators': [25, 50, 100],
        'criterion': ['gini', 'entropy'],
        'max_depth': [10, 25, 50, None]
    }
}

# LogisticRegression
clfs['LogisticRegression'] = {
    'instance': LogisticRegression(max_iter=500, n_jobs=-1),
    'params': [
            {'penalty': ['l2'], 'C': [0.1, 0.5, 1, 5, 10]},
            {'penalty': ['l1'], 'solver': ['liblinear', 'saga'], 'C': [0.1, 0.5, 1, 5, 10]},
            {'penalty': ['elasticnet'], 'C': [0.1, 0.5, 1, 5, 10], 'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]}
        ]
}

In [ ]:
for clf_name, clf in clfs.items():
    print('<{}>'.format(clf_name))
    print('  training ...'.format(clf_name))
    
    gs = GridSearchCV(clf['instance'], param_grid=clf['params'], cv=5, n_jobs=-1)
    gs.fit(X, y)
    clfs[clf_name]['best_estimator'] = gs.best_estimator_
    
    print('  best_score: {:.3f}'.format(gs.best_score_))
    print('  best_params: {}'.format(gs.best_params_))

In [ ]:
# 各モデルの予測結果から多数決を採用するソフト分類器を作成
vote = VotingClassifier(
    estimators=[(clf_name, clf['best_estimator']) for clf_name, clf in clfs.items()], 
    voting='soft',
    n_jobs=-1
)
vote.fit(X, y)
vote.estimators_

In [ ]:
# 各モデルとソフト分類器で、予測結果を比較する
# randomforestだけ精度が異常に高いのは、多分過学習してる。。。
for clf_name, clf in clfs.items():
    score = accuracy_score(y, clf['best_estimator'].predict(X))
    print(clf_name, score)
print('Vote', accuracy_score(y, vote.predict(X)))

In [ ]:
# 各モデル個別の予測確率と、全モデルを集約した分類器の予測確率を可視化
# 今回の分類器はソフト投票（各モデルの予測結果の確率の平均をとる）のため、個別のモデルよりもなだらかな分布になっている。
predict_proba = pd.DataFrame(
    {clf_name: clf['best_estimator'].predict_proba(X)[:, 1] for clf_name, clf in clfs.items()},
    index=X.index)
predict_proba['Vote'] = vote.predict_proba(X)[:, 1]
_ = predict_proba.plot(kind='hist', bins=50, grid=True, alpha=0.5, figsize=(16,8))

In [ ]:
# 各モデルの予測確率を予測結果ファイルとして出力
columns = predict_proba.columns
for column in columns:
    sub[column] = 0.5

mask = [idx for idx in sub.index if idx in X.index]
sub.loc[mask, columns] = predict_proba.loc[mask, columns]

for column in columns:
    sub[['ID', column]].rename(columns={column: 'pred'}).to_csv('predict_proba-{}.csv'.format(column), index=False)

In [ ]:
# 各モデルの勝敗（0か1）を予測結果ファイルとして出力
predict = pd.DataFrame(
    {clf_name: clf['best_estimator'].predict(X) for clf_name, clf in clfs.items()},
    index=X.index)
predict['Vote'] = vote.predict(X)

columns = predict.columns
for column in columns:
    sub[column] = 0.5
    
mask = [idx for idx in sub.index if idx in X.index]
sub.loc[mask, columns] = predict.loc[mask, columns]

for column in columns:
    sub[['ID', column]].rename(columns={column: 'pred'}).to_csv('predict-{}.csv'.format(column), index=False)

In [ ]:
import shutil
target_name = 'predict_proba-RandomForestClassifier.csv'
new_name = 'final-submission.csv'
shutil.copy(target_name, new_name)